In [1]:
import h5py
import numpy as np
import pandas as pd
import tables
import os
import sys
import time
import glob
import scipy.io as sio
import hdf5_getters
from pandas import read_hdf
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

from sklearn.metrics import mean_squared_error
import matplotlib as plt

Using TensorFlow backend.


In [3]:
finalWorkingDF = pd.read_pickle("finalWorkingDF")

finalWorkingDF = finalWorkingDF[finalWorkingDF.year != 0]
finalWorkingDF.fillna(finalWorkingDF.mean(), inplace=True)

classifierData = finalWorkingDF
classifierData['year'] = np.floor(classifierData['year']/10)*10


In [4]:
# Separate data into training and testing set
y = finalWorkingDF['year']
x = finalWorkingDF.drop('year', axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

y_classify = finalWorkingDF['year']
x_classify = finalWorkingDF.drop('year', axis = 1)
x_classify_train, x_classify_test, y_classify_train, y_classify_test = train_test_split(x_classify, y_classify, test_size=0.2)
classes = np.unique(classifierData['year'])

In [5]:
# x_train.to_pickle("XTrain")
# y_train.to_pickle("YTrain")
# x_test.to_pickle("XTest")
# y_test.to_pickle("YTest")

In [27]:
###################### REGRESSION NEURAL NETWORK ########################
# nCols = x_train.shape[1]
# # create model
# model = Sequential()
# #model.add(Dense(12, input_shape=(3744,37), activation='relu'))
# model.add(Dense(12, input_shape=(nCols,), activation='relu'))
# model.add(Dense(8, activation='relu'))
# model.add(Dense(1, activation='relu'))

# # Compile model
# model.compile(loss='mean_squared_error', optimizer='adam')

# # Fit the model
# model.fit(x_train.to_numpy(), y_train.to_numpy(), epochs=150, batch_size=10)

# # evaluate the model
# train_scores = model.evaluate(x_train, y_train)
# #print("\n%s: %.2f%%" % (model.metrics_names[1], train_scores[1]*100))
# test_scores = model.evaluate(x_test, y_test)
# #print("\n%s: %.2f%%" % (model.metrics_names[1], test_scores[1]*100))

# ###################### Neural Network Analysis ########################
# print(model.evaluate(x_train, y_train))
# print(model.evaluate(x_test, y_test))

Epoch 1/150
3744/3744 [==============================] - 1s 180us/step - loss: 1566414.6791
Epoch 2/150
3744/3744 [==============================] - 0s 85us/step - loss: 188602.3279
Epoch 3/150
3744/3744 [==============================] - 0s 81us/step - loss: 111722.8118
Epoch 4/150
3744/3744 [==============================] - 0s 81us/step - loss: 78767.6423
Epoch 5/150
3744/3744 [==============================] - 0s 81us/step - loss: 65061.8613
Epoch 6/150
3744/3744 [==============================] - 0s 80us/step - loss: 56789.2227
Epoch 7/150
3744/3744 [==============================] - 0s 79us/step - loss: 50722.9243
Epoch 8/150
3744/3744 [==============================] - 0s 81us/step - loss: 45956.7669
Epoch 9/150
3744/3744 [==============================] - 0s 83us/step - loss: 41905.0038
Epoch 10/150
3744/3744 [==============================] - 0s 81us/step - loss: 39184.2753
Epoch 11/150
3744/3744 [==============================] - 0s 92us/step - loss: 37106.8225
Epoch 12/150
3

3744/3744 [==============================] - 0s 83us/step - loss: 14669.1083
Epoch 93/150
3744/3744 [==============================] - 0s 83us/step - loss: 14291.2855
Epoch 94/150
3744/3744 [==============================] - 0s 88us/step - loss: 14194.5317
Epoch 95/150
3744/3744 [==============================] - 0s 85us/step - loss: 14304.5152
Epoch 96/150
3744/3744 [==============================] - 0s 90us/step - loss: 13936.9103
Epoch 97/150
3744/3744 [==============================] - 0s 84us/step - loss: 13985.3870
Epoch 98/150
3744/3744 [==============================] - 0s 85us/step - loss: 13799.8466
Epoch 99/150
3744/3744 [==============================] - 0s 87us/step - loss: 13707.6038
Epoch 100/150
3744/3744 [==============================] - 0s 91us/step - loss: 13640.3626
Epoch 101/150
3744/3744 [==============================] - 0s 89us/step - loss: 13524.8909
Epoch 102/150
3744/3744 [==============================] - 0s 92us/step - loss: 13422.9098
Epoch 103/150
3744/3

In [68]:
###################### CLASSIFICATION NEURAL NETWORK ########################
# import keras
# from keras.layers import Dense, Dropout, Activation
# from keras.optimizers import SGD
# from sklearn.preprocessing import LabelEncoder
# from keras.utils import np_utils

# # encode class values as integers
# encoderTrain = LabelEncoder()
# encoderTrain.fit(y_classify_train)
# encoderTrainY = encoder.transform(y_classify_train)
# # convert integers to dummy variables (i.e. one hot encoded)
# dummy_y_train = np_utils.to_categorical(encoderTrainY)

# # encode class values as integers
# encoderTest = LabelEncoder()
# encoderTest.fit(y_classify_test)
# encoderTestY = encoder.transform(y_classify_test)
# # convert integers to dummy variables (i.e. one hot encoded)
# dummy_y_test = np_utils.to_categorical(encoderTestY)

# nCols = x_classify_train.shape[1]


# model.add(Dense(64, activation='relu', input_dim=nCols))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(10, activation='softmax'))

# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='categorical_crossentropy',
#               optimizer=sgd,
#               metrics=['accuracy'])

# model.fit(x_classify_train.to_numpy(), dummy_y_train, epochs=20, batch_size=128)
# score = model.evaluate(x_classify_test.to_numpy(), dummy_y_test, batch_size=128)
# print(score)
# print(neural_network.predict(x_classify_test.to_numpy()[0:10], batch_size=32, verbose=0))


Epoch 1/20
3744/3744 [==============================] - 1s 250us/step - loss: 1.8655 - acc: 0.4722
Epoch 2/20
3744/3744 [==============================] - 0s 27us/step - loss: 1.4014 - acc: 0.5422
Epoch 3/20
3744/3744 [==============================] - 0s 29us/step - loss: 1.3558 - acc: 0.5427
Epoch 4/20
3744/3744 [==============================] - 0s 26us/step - loss: 1.3387 - acc: 0.5433
Epoch 5/20
3744/3744 [==============================] - 0s 26us/step - loss: 1.3242 - acc: 0.5430
Epoch 6/20
3744/3744 [==============================] - 0s 26us/step - loss: 1.3228 - acc: 0.5430
Epoch 7/20
3744/3744 [==============================] - 0s 27us/step - loss: 1.3145 - acc: 0.5430
Epoch 8/20
3744/3744 [==============================] - 0s 27us/step - loss: 1.3134 - acc: 0.5430
Epoch 9/20
3744/3744 [==============================] - 0s 27us/step - loss: 1.3077 - acc: 0.5430
Epoch 10/20
3744/3744 [==============================] - 0s 27us/step - loss: 1.3031 - acc: 0.5430
Epoch 11/20
3744/3

**10-701 Song Project**

**Introduction**
For this project, we are exploring the capabilities of musical factors and song lyrics the year a song was released. In particular we are looking at the 10,000 song subset of the million song dataset and the musixmatch dataset for the 10,000 song subset. The 10,000 song subset contains very detailed information about each song such as timbre and pitch vectors for each segment of the song, tatums, loudness and various other algorithmically determined factors such as artist hotness, song hotness, danceability, etc. In contrast, the musixmatch dataset only contains bag-of-words lyrics for each song. Our goal is to determine whether models predicting a song's release year trained on the musixmatch dataset can exhibit similar testing performance as models trained on the full feature-set of the 10k subset.

**Background/Literature**


## Methods/Model

### Preliminary Data Analysis and Feature Extraction

We used a 10,000 song subset of the Million Song Dataset for our analysis, since the million songs would have taken much to long to run and was too large to store locally on our computers. We downloaded the 10,000 song subset which had the data for each song represented as a HDF5 file. We scripted through this dataset, extracting out each songs features and converting the data into a pandas dataframe. We then removed every song in the dataframe that had a year value of 0, which indicated that the database did not have the year recorded for that song. In order to make our predictive models that used the musical qualities of a song to predit the year it was released, we dropped features from the dataframe that were unrelated to the musical qualities for the songs such as "artist_familiarity", "artist_location", "track_id", "artist_id", etc. Confidence values for song features were also included in the dataset such as "bars_confidence", "beats_confidence","key_confidence", etc. which we also decided to remove, however the impacts of retaining these variables will be examined in the final paper. The dataframe also included two-dimensional arrays that listed 12 dimensional timbre and pitch vectors at each point in the song. In order to make these two-dimensional vectors readable to our machine learning algorithms, we took the weighted average of the 12 dimensional timbre and pitch vectors based on how long that pitch/timbre lasted before the onset of another pitch/timbre vector given by the "segments_start" variable. We then set each value in new 12-dimensional timbre and pitch vectors to represent a new feature, so we now had 12 features for timbre and 12 for pitch for every song. 

As you can see though by displaying the covariances between each of the variables, the transformed pitch values are very highly correlated with each of the corresponding transformed timbre values.


In [ ]:
###################### Covariance Analysis ########################
from yellowbrick.features import Rank2D

visualizer = Rank2D(algorithm="spearman")
visualizer.fit_transform(x)
visualizer.poof()

However, when graphing the Pitch1 and Timber1 values there does not seem to be that much of a relation between them.

In [ ]:
###################### More Covariance Analysis Between Pitch1 and Timber1 ########################

from yellowbrick.features import JointPlotVisualizer

visualizer = JointPlotVisualizer(feature='Tim1', target='Pitch1')
visualizer.fit(x['Tim1'], x['Pitch1'])
visualizer.poof()

We also remove the "sections_start" and "tatums_start" variables in the dataset which represented the times where the largest grouping and smallest grouping respectively in a song would begin because we only used the weighted average of the pitches and timbre vectors, so this information would not be useful. One of the most difficult aspects of this project was examining the data and seeing which predictors to include in our models and how we shoould include certain variables in thee model. This is what makes using lyrics to predict year instead much more simple, which was the impetus for this project. We also hope to examine using the song information differently such as using the "sections_start" and "tatums_start" to extract out the most important timbre and pitch vectors in a song. In addition, when a song does not have information for a certain variable, we input the mean of that feature, so that we don't throw out so much data since there are so many variables that could be empty.

### Preliminary Models

We used extracted data to create k nearest neighbor, random forest, regularized logistic regression, support vector machine, and naive bayes classifiers that predict the decade a song came out.

We also build k nearest neighbor, ridge and lasso linear regression to attempt and get the exact year a song was released.


In [ ]:
###################### KNN CLASSIFIER ########################
n_neighbors = 15

nnClassifier = neighbors.KNeighborsClassifier(n_neighbors)
nnClassifier.fit(x_classify_train, y_classify_train)

# Get training and test predictions
predictedtrainyears = nnClassifier.predict(x_classify_train)
trainMSE = mean_squared_error(y_classify_train, predictedtrainyears)
testMSE = mean_squared_error(y_classify_test, nnClassifier.predict(x_classify_test))
# classifier_trainMSE = np.mean((nnClassifier.predict(x_trainNumPy) - y_trainNumPy)**2)
# classifier_testMSE = np.mean((nnClassifier.predict(x_testNumPy) - y_testNumPy)**2)

print("training MSE", trainMSE)
print("testing MSE", testMSE)

In [ ]:
###################### KNN REGRESSION ########################
n_neighbors = 15

nnRegressor = neighbors.KNeighborsRegressor(n_neighbors)
nnRegressor.fit(x_train, y_train)

# Get training and test predictions
predictedtrainyears = nnRegressor.predict(x_train)
trainMSE = mean_squared_error(y_train, predictedtrainyears)
testMSE = mean_squared_error(y_test, nnRegressor.predict(x_test))

print("training MSE", trainMSE)
print("testing MSE", testMSE)

In [ ]:
###################### RIDGE LINEAR REGRESSION (REGULARIZER FIT WITH CV)########################
from sklearn import linear_model
regRidge = linear_model.RidgeCV(alphas=[0.1, 1.0, 10.0], cv=3)
regRidge.fit(x_train, y_train)
RidgeCV(alphas=[0.1, 1.0, 10.0], cv=3, fit_intercept=True, scoring=None, normalize=False)
regRidge.alpha_

In [ ]:
###################### LASSO LINEAR REGRESSION ########################

regLasso = linear_model.Lasso(alpha=0.1)
regLasso.fit(x_train, y_train)
Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
regLasso.alpha_

In [ ]:
###################### Residuals by Prediction (KNN) ########################
from yellowbrick.regressor import ResidualsPlot
visualizer = ResidualsPlot(nnRegressor)
#visualizer.fit(x_train, y_train)
visualizer.score(x_test, y_test)
visualizer.poof()

In [ ]:
###################### Residuals by Prediction (Linear RIDGE) ########################
from yellowbrick.regressor import ResidualsPlot
visualizer = ResidualsPlot(regRidge)
#visualizer.fit(x_train, y_train)
visualizer.score(x_test, y_test)
visualizer.poof()

In [ ]:
###################### Residuals by Prediction (Linear LASSO) ########################
from yellowbrick.regressor import ResidualsPlot
visualizer = ResidualsPlot(regLasso)
#visualizer.fit(x_train, y_train)
visualizer.score(x_test, y_test)
visualizer.poof()

In [2]:
###################### LOGISTIC REGRESSION ########################

#LogisticRegression(multi_class='multinomial',solver ='newton-cg').fit(x_classify_train, y_classify_train)
from sklearn.linear_model import LogisticRegression
X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(x_classify_train, y_classify_train)
clf.predict(x_classify_test)
clf.predict_proba(x_classify_test) 
clf.score(x_classify_test, y_classify_test)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [ ]:
###################### SUPPORT VECTOR MACHINE MODELS ########################

# RBF Kernel
clf = svm.SVC(gamma='scale', decision_function_shape='ovo')
clf.fit(x_classify_test, y_classify_test) 
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

# Linear Kernel
lin_clf = svm.LinearSVC()
lin_clf.fit(X, Y) 
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

# Apparently it also SVMs also do regression 
clf = svm.SVR()
clf.fit(x_train, y_train) 
SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)
clf.predict(x_test, y_test)

In [ ]:
###################### Classifier Information for Gaussian Naive Bayes ########################

from sklearn.naive_bayes import GaussianNB
from yellowbrick.classifier import ClassificationReport

# Instantiate the classification model and visualizer
bayes = GaussianNB()
visualizer = ClassificationReport(bayes, classes=classes, support=True)
visualizer.fit(x_classify_train, y_classify_train)  # Fit the visualizer and the model
visualizer.score(x_classify_test, y_classify_test)  # Evaluate the model on the test data
g = visualizer.poof()             # Draw/show/poof the data

In [ ]:
###################### Classifier Probabilities for Random Forest ########################

from sklearn.ensemble import RandomForestClassifier

from yellowbrick.classifier import ClassPredictionError

# Instantiate the classification model and visualizer
visualizer = ClassPredictionError(
    RandomForestClassifier(), classes=classes
)

# Fit the training data to the visualizer
visualizer.fit(x_classify_train, y_classify_train)

# Evaluate the model on the test data
visualizer.score(x_classify_test, y_classify_test)

# Draw visualization
g = visualizer.poof()

In [ ]:
###################### ROC Curves for Random Forest ########################

import yellowbrick

visualizer = yellowbrick.classifier.rocauc.ROCAUC(RandomForestClassifier(), classes=classes)

visualizer.fit(x_classify_train, y_classify_train)  # Fit the training data to the visualizer
visualizer.score(x_classify_test, y_classify_test)  # Evaluate the model on the test data
g = visualizer.poof()             # Draw/show/poof the data

In [ ]:
############## YELLOWBRICK ALSO HAS GOOD STUFF FOR DISPLAYING INFO ABOUT TEXTS #################